# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:
from azureml.core import Workspace, Experiment
from azureml.train.automl import AutoMLConfig
from azureml.data.dataset_factory import TabularDatasetFactory
from azureml.core import Dataset
from azureml.widgets import RunDetails

## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.


TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [2]:
ws = Workspace.from_config()
experiment_name = 'capstoneexperiment'
experiment=Experiment(ws, experiment_name)

ds = Dataset.get_by_name(ws,'alcohol')
ds.to_pandas_dataframe()

,school,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,Fjob,...,famrel,freetime,goout,Dalc,Walc,health,absences,G1,G2,G3
0,GP,F,18,U,GT3,A,4,4,at_home,teacher,...,4,3,4,1,1,3,6,5,6,6
1,GP,F,17,U,GT3,T,1,1,at_home,other,...,5,3,3,1,1,3,4,5,5,6
2,GP,F,15,U,LE3,T,1,1,at_home,other,...,4,3,2,2,3,3,10,7,8,10
3,GP,F,15,U,GT3,T,4,2,health,services,...,3,2,2,1,1,5,2,15,14,15
4,GP,F,16,U,GT3,T,3,3,other,other,...,4,3,2,1,2,5,4,6,10,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
390,MS,M,20,U,LE3,A,2,2,services,services,...,5,5,4,4,5,4,11,9,9,9
391,MS,M,17,U,LE3,T,3,1,services,services,...,2,4,5,3,4,2,3,14,16,16
392,MS,M,21,R,GT3,T,1,1,other,other,...,5,5,3,3,3,3,3,10,8,7
393,MS,M,18,R,LE3,T,3,2,services,other,...,4,4,1,3,4,5,0,11,12,10


## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

In [3]:
automl_settings = {"experiment_timeout_minutes":30, "primary_metric":'accuracy', "n_cross_validations":2}

automl_config = AutoMLConfig(
    task='classification',
    training_data=ds,
    label_column_name='Walc', **automl_settings
    )

In [4]:
# TODO: Submit your experiment
remote_run = experiment.submit(automl_config)

## Run Details
Here  are  some examples of other models tried by  the experiment:

Model: MaxAbsScaler   Acuracy: 0.4

Model:: StandardScalerWraper Acuracy: 0.28

Model:: ExtrmeRandomTrees: 0.28

BBut aas eexpeectteed, the  VVttiing  EEnnsemmblle  Model takes the best moddels and  furthermore  it combines them. That is why no other methods are as effective.

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [5]:
RunDetails(remote_run).show()

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [6]:
best_model, fitted_model = remote_run.get_output()
best_model.get_tags()

{'ensembled_iterations': '[32, 27, 34, 46, 64, 20, 26, 39, 51, 63]',
 'ensembled_algorithms': "['ExtremeRandomTrees', 'RandomForest', 'ExtremeRandomTrees', 'ExtremeRandomTrees', 'GradientBoosting', 'LightGBM', 'ExtremeRandomTrees', 'RandomForest', 'RandomForest', 'RandomForest']",
 'ensemble_weights': '[0.13333333333333333, 0.06666666666666667, 0.06666666666666667, 0.06666666666666667, 0.06666666666666667, 0.06666666666666667, 0.3333333333333333, 0.06666666666666667, 0.06666666666666667, 0.06666666666666667]',
 'best_individual_pipeline_score': '0.5038327436804594',
 'best_individual_iteration': '32',
 'model_explanation': 'True'}

In [7]:
#TODO: Save the best model
import joblib

parameter_values = best_model.get_details()
os.makedirs('outputs',exist_ok=True)
filename = 'finalized_model.sav'
modeldoc = joblib.dump('outputs/model.joblib', filename)
print(modeldoc)

['finalized_model.sav']


## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [8]:
from azureml.core import Model
from azureml.core.model import Model
from azureml.core import Environment
from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice, Webservice

model=remote_run.register_model()

ent = best_model.download_file('outputs/scoring_file_v_1_0_0.py','score.py')
env = Environment.get(workspace=ws,name='AzureML-AutoML')


inference_config = InferenceConfig(entry_script='score.py', environment=env)

aci_config = AciWebservice.deploy_configuration(cpu_cores = 1,
                                                       memory_gb = 1,enable_app_insights=True,auth_enabled=True)

service = Model.deploy(workspace = ws,
                       name = "capstonewebservice4",
                       models = [model],
                       inference_config = inference_config,
                       deployment_config = aci_config)
service.wait_for_deployment(True)
print(service.state)


Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running..................................................
Succeeded
ACI service creation operation finished, operation "Succeeded"
Healthy


In [9]:
print(remote_run.model_id)

AutoML963efbeb067


In [10]:
print(service.scoring_uri)
print(service.swagger_uri)

http://465cb530-a01d-468c-9e73-98da8c3bad6a.southcentralus.azurecontainer.io/score
http://465cb530-a01d-468c-9e73-98da8c3bad6a.southcentralus.azurecontainer.io/swagger.json


In [12]:
primary, secondary = service.get_keys()
print(primary)

uKTfanUhLztZgg2GA7ifxoleJ6rxlWlo


TODO: In the cell below, send a request to the web service you deployed to test it.

In [14]:
import requests
import json

# URL for the web service
scoring_uri = 'http://465cb530-a01d-468c-9e73-98da8c3bad6a.southcentralus.azurecontainer.io/score'
# If the service is authenticated, set the key or token
key = 'uKTfanUhLztZgg2GA7ifxoleJ6rxlWlo'

# Two sets of data to score, so we get two results back
data = {"data":
        [
            [
              'MS', 'F', 19, 'R', 'GT3', 'T', 2, 3, 'services', 'other','other', 'mother', 1, 3, 1, 'no', 'no', 'no', 'yes', 'no', 'yes', 'yes', 'no', 5, 4, 2, 1, 5, 0, 7, 5, 0
            ],
           [
              'MS', 'M', 19, 'U', 'GT3', 'T', 2, 3, 'services', 'other', 'teacher','other', 1, 3, 1, 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 1, 4, 2, 1, 5, 0, 7, 5, 0
            ]
        ]
        }
# Convert to JSON string
input_data = json.dumps(data)

# Set the content type
headers = {'Content-Type': 'application/json'}
# If authentication is enabled, set the authorization header
headers['Authorization'] = f'Bearer {key}'

# Make the request and display the response
resp = requests.post(scoring_uri, input_data, headers=headers)
print(resp.text)

"{\"result\": [1, 1]}"


TODO: In the cell below, print the logs of the web service and delete the service

In [15]:
service.get_logs()

'2021-03-04T18:48:03,515756000+00:00 - iot-server/run \n2021-03-04T18:48:03,528202700+00:00 - gunicorn/run \n2021-03-04T18:48:03,555425400+00:00 - rsyslog/run \nrsyslogd: /azureml-envs/azureml_661474bbe74e96b5d8added5888dfc85/lib/libuuid.so.1: no version information available (required by rsyslogd)\n2021-03-04T18:48:03,621457200+00:00 - nginx/run \n/usr/sbin/nginx: /azureml-envs/azureml_661474bbe74e96b5d8added5888dfc85/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)\n/usr/sbin/nginx: /azureml-envs/azureml_661474bbe74e96b5d8added5888dfc85/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)\n/usr/sbin/nginx: /azureml-envs/azureml_661474bbe74e96b5d8added5888dfc85/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)\n/usr/sbin/nginx: /azureml-envs/azureml_661474bbe74e96b5d8added5888dfc85/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)\n/usr/sbin/nginx

In [ ]:
service.delete()